In [19]:
import pandas as pd
import numpy as np
from modules import DataDrop as DD

GROUP_NAME = "highcc_one_season_group"
PATH_READ = "./Dataset/" + GROUP_NAME + "/Train_Test.csv"
PATH_SAVE_TRAIN = "./Dataset/" + GROUP_NAME + "/train.csv"
PATH_SAVE_TEST =  "./Dataset/" + GROUP_NAME + "/test.csv"
PATH_SAVE_TEST_PLANB = "./Dataset/" + GROUP_NAME + "/testB.csv"
PATH_SAVE_TEST_PLANB_INDEX = "./Dataset/" + GROUP_NAME + "/testB_index.csv"
PATH_WEATHER = "./raw_data/merged_weatherdata.csv"

In [20]:
raw_dataset = pd.read_csv(PATH_READ)
df_weather = pd.read_csv(PATH_WEATHER)
df_weather = DD.weather_drop(df_weather)
dataset = raw_dataset.copy()


dataset["supplied_at"] = pd.to_datetime(dataset["supplied_at"], format="%Y-%m-%d %X")


train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)


In [21]:
df_weather.set_index("date", inplace=True)

In [22]:
test_dataset = test_dataset.reset_index(drop=True)

In [23]:
test_data_planB = {}
test_index_planB = {}
first_index = 0
final_index = 0



In [24]:
for i in range(len(test_dataset)):
    test_data_planB[i] = df_weather.loc[test_dataset.loc[i, "last_supply_time"]: test_dataset.loc[i, "supplied_at"]]
    test_data_planB[i].reset_index(inplace=True)
    test_data_planB[i].drop(columns = ["index"], inplace = True)
    test_data_planB[i]["tank_type_id"] = test_dataset.loc[i, "tank_type_id"]
    test_data_planB[i]["customer_id"] = test_dataset.loc[i, "customer_id"]
    test_data_planB[i]["last_year_mean"] = test_dataset.loc[i, "last_year_mean"]
    test_data_planB[i]["last_year_sum"] = test_dataset.loc[i, "last_year_sum"]
    test_data_planB[i]["price"] = test_dataset.loc[i, "price"]
    first_index = final_index
    final_index += len(test_data_planB[i])
    test_index_planB[i] = [test_dataset.loc[i, "customer_id"], test_dataset.loc[i, "volume"], first_index, final_index-1]

df_testb = pd.concat(test_data_planB)
df_testb_index = pd.DataFrame.from_dict(test_index_planB, orient='index',
                       columns=['ID', 'volume', 'first_index', 'final_index'])



c:\Users\zhaobang\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\zhaobang\AppData\Local\Temp/ipykernel_6188/2142747265.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data_planB[i]["tank_type_id"] = test_dataset.loc[i, "tank_type_id"]
C:\Users\zhaobang\AppData\Local\Temp/ipykernel_6188/2142747265.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [7]:
def clear_test_train_data(dataset):
    dataset.pop("customer_id")
    dataset.pop("tank_type_id")
    dataset.pop("sum")
    dataset.pop("mean")
    dataset.pop("last_supply_time")
    date_time = dataset.pop("supplied_at")
    timestamp_s = date_time.map(pd.Timestamp.timestamp)
    day = 24*60*60
    year = (365.2425)*day

    dataset['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
    dataset['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
    return dataset


def clear_testb_data(dataset):
    dataset.pop("customer_id")
    dataset.pop("tank_type_id")
    date_time = dataset.pop("date")
    timestamp_s = date_time.map(pd.Timestamp.timestamp)
    day = 24*60*60
    year = (365.2425)*day

    dataset['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
    dataset['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
    return dataset

In [8]:
train = clear_test_train_data(train_dataset)
test = clear_test_train_data(test_dataset)
testb = clear_testb_data(df_testb)
testb["interval"] = 1

In [9]:
test.head()

,volume,matemp,mltemp,mhtemp,mwind,msnowacc,msnow,msun,mhumidi,mean_consumption,interval,last_year_mean,last_year_sum,price,Year sin,Year cos
0,224.0,-2.093103,0.979310,-5.058621,2.931034,16.137931,2.275862,3.272414,69.172414,7.724138,29.0,9.176761,1303.1,68,-0.132823,0.991140
1,285.8,2.842857,5.903571,0.085714,3.192857,52.821429,3.000000,3.385714,65.285714,10.207143,28.0,9.176761,1303.1,57,0.737751,0.675073
2,305.3,3.193548,6.390323,0.370968,2.858065,51.000000,4.258065,3.348387,65.387097,9.848387,31.0,9.441667,1359.6,85,0.576903,0.816812
3,239.8,3.666667,6.635714,0.909524,3.690476,53.214286,3.309524,3.530952,67.690476,5.709524,42.0,8.222727,1266.3,89,0.740498,0.672058
4,185.7,4.161905,8.004762,0.600000,3.423810,72.523810,4.666667,3.090476,68.952381,8.842857,21.0,7.927612,1062.3,104,0.683443,0.730004


In [10]:
testb.head()

atemperature  ltemperature  sunshine  sonw(accumulated)  htemperature  \
0 0          -1.6          -6.9       2.4                 35           2.4   
  1           2.8           0.9       1.4                 28           4.1   
  2           2.2           0.1       1.6                 17           4.5   
  3           2.3          -0.1       6.0                 15           4.5   
  4           1.7          -2.3       4.9                 13           5.5   

     snow  windSpeed(m/s)  humidity(%)  last_year_mean  last_year_sum  price  \
0 0     1             1.9           75        9.176761         1303.1     68   
  1     0             3.1           85        9.176761         1303.1     68   
  2     2             4.0           81        9.176761         1303.1     68   
  3     0             2.4           60        9.176761         1303.1     68   
  4     0             2.8           73        9.176761         1303.1     68   

     Year sin  Year cos  interval  
0 0 -0.582634  0.812734         1  
  1 -0.568568  0.822637         1  
  2 -0.554333  0.832295         1  
  3 -0.539933  0.841708         1  
  4 -0.525375  0.850871         1

In [40]:
# testb = testb[["atemperature", "ltemperature", "htemperature", "windSpeed(m/s)", "sonw(accumulated)", "snow", "sunshine", "humidity(%)", ""]]

In [11]:
cols = testb.columns.tolist()

In [12]:
cols = cols[:2] + [cols[4]] + [cols[6]] + [cols[3]] + [cols[5]] + [cols[2]] + [cols[7]] + [cols[len(cols)-1]] + cols[-6:len(cols)-1]

In [13]:
colsa = test.columns.tolist()

In [14]:
colsa

['volume',
 'matemp',
 'mltemp',
 'mhtemp',
 'mwind',
 'msnowacc',
 'msnow',
 'msun',
 'mhumidi',
 'mean_consumption',
 'interval',
 'last_year_mean',
 'last_year_sum',
 'price',
 'Year sin',
 'Year cos']

In [15]:
cols

['atemperature',
 'ltemperature',
 'htemperature',
 'windSpeed(m/s)',
 'sonw(accumulated)',
 'snow',
 'sunshine',
 'humidity(%)',
 'interval',
 'last_year_mean',
 'last_year_sum',
 'price',
 'Year sin',
 'Year cos']

In [28]:
testb = testb[cols]

In [29]:
train.to_csv(PATH_SAVE_TRAIN, index=False)
test.to_csv(PATH_SAVE_TEST, index=False)
testb.to_csv(PATH_SAVE_TEST_PLANB, index=False)
df_testb_index.to_csv(PATH_SAVE_TEST_PLANB_INDEX, index=False)